In [ ]:
!pip -q install transformers accelerate safetensors bitsAndBytes


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStremer
import torch

repo = "Luke-griggs/small-talk-1.3"

tokenizer = AutoTokenizer.from_pretrained(repo)
model = AutoModelForCausalLM.from_pretrained(
    repo,
    torch_dtype=torch.float16,
    device_map="auto"
    )
model.eval()


In [ ]:
test_prompt1 = "what is your favorite thing to do?"
test_prompt2 = "what is a recent accomplishment you're proud of?"
test_prompt3 = "what is a place that you've always wanted to go?"
test_prompt4 = "when was the last time you felt excited about something?"

test_prompt = test_prompt4

chat = [
    {"role": "system", "content": "You are a helpful, polite, and friendly assistant."},
    {"role": "user", "content": test_prompt}
]

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True).to(device)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )
